In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [12]:
def unet(input_size=(None, None, None, 1)):
    inputs = Input(input_size)
    conv1 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    conv4 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv4)

    up5 = concatenate([UpSampling3D(size=(2, 2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling3D(size=(2, 2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling3D(size=(2, 2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(conv7)

    conv8 = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=[inputs], outputs=[conv8])
    model.summary()
    # model.compile(optimizer = Adam(lr = 1e-4),
    # loss = cross_entropy_balanced, metrics = ['accuracy'])
    return model

In [13]:
def data_norm(image):
    mean = np.mean(image)
    std = np.std(image)
    return (image - mean) / std

In [14]:
def show_history(history):
    # list all data in history
    print(history.history.keys())
    plt.figure(figsize=(10, 6))
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy', fontsize=20)
    plt.ylabel('Accuracy', fontsize=20)
    plt.xlabel('Epoch', fontsize=20)
    plt.legend(['train', 'test'], loc='center right', fontsize=20)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.tick_params(axis='both', which='minor', labelsize=18)
    plt.show()

    # summarize history for loss
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss', fontsize=20)
    plt.ylabel('Loss', fontsize=20)
    plt.xlabel('Epoch', fontsize=20)
    plt.legend(['train', 'test'], loc='center right', fontsize=20)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.tick_params(axis='both', which='minor', labelsize=18)
    plt.show()

In [15]:
def load_sample(file_path, shape=(128, 128, 128), dtype=np.single, norm=True):
    sample = np.fromfile(file_path, dtype=dtype).reshape(shape)
    if norm:
        sample = data_norm(sample)
    # In seismic processing, the dimensions of a seismic array is often arranged as
    # a[n3][n2][n1] where n1 represents the vertical dimension.
    return np.transpose(sample)


class DataGenerator(Sequence):
    """Generates data for keras"""

    def __init__(self, path, ids, batch_size=1, shape=(128, 128, 128),
                 n_channels=1, shuffle=True):
        """Initialization"""
        self.shape = shape
        self.path = path
        self.batch_size = batch_size
        self.ids = ids
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        """Generates one batch of data"""
        # Generate indexes of the batch
        bsize = self.batch_size
        indexes = self.indexes[index * bsize:(index + 1) * bsize]

        # Find list of IDs
        ids = [self.ids[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(ids)

        return X, Y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, ids):
        """Generates data containing batch_size samples"""
        # Initialization
        seis = load_sample(f'{self.path}/seis/{ids[0]}.dat', norm=True)
        fault = load_sample(f'{self.path}/fault/{ids[0]}.dat', norm=False)
        seis = np.reshape(seis, self.shape)

        # Generate data
        X = np.zeros((2, *self.shape, self.n_channels), dtype=np.single)
        Y = np.zeros((2, *self.shape, self.n_channels), dtype=np.single)
        X[0,] = np.reshape(seis, (*self.shape, self.n_channels))
        Y[0,] = np.reshape(fault, (*self.shape, self.n_channels))
        X[1,] = np.reshape(np.flipud(seis), (*self.shape, self.n_channels))
        Y[1,] = np.reshape(np.flipud(fault), (*self.shape, self.n_channels))
        '''
        for i in range(4):
          X[i,] = np.reshape(np.rot90(seis,i,(2,1)), (*self.shape,self.n_channels))
          Y[i,] = np.reshape(np.rot90(fault,i,(2,1)), (*self.shape,self.n_channels))
        '''
        return X, Y


In [16]:
# input image dimensions
params = {'batch_size': 1,
          'shape': (128, 128, 128),
          'n_channels': 1,
          'shuffle': True}
train_path = "./data/train/"
val_path = "./data/validation/"

train_ids = range(len(os.listdir(train_path)))
val_ids = range(len(os.listdir(val_path)))
train_generator = DataGenerator(path=train_path,
                                ids=train_ids, **params)
valid_generator = DataGenerator(path=val_path,
                                ids=val_ids, **params)
model = unet(input_size=(None, None, None, 1))
model.compile(optimizer=Adam(lr=1e-4), loss='bce',
              metrics=['accuracy'])
model.summary()

# checkpoint
filepath = "model/model-{epoch:02d}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc',
                             verbose=1, save_best_only=False, mode='max')
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                              patience=20, min_lr=1e-8)
callbacks_list = [checkpoint]
print("data prepared, ready to train!")
# Fit the model
history = model.fit(train_generator,
                    validation_data=valid_generator, epochs=1, callbacks=callbacks_list, verbose=1)
model.save('model/model.h5')
show_history(history)

SyntaxError: invalid syntax (2468655728.py, line 15)